# Level UP your Python

### Henry Schreiner

In [ ]:
%config InteractiveShell.ast_node_interactivity="last_expr_or_assign"

# Could enable pretty tracebacks (ONLY DO IN APPLICATION, NOT LIBRARY):
# import rich.traceback
# rich.traceback.install(show_locals=True)
#
# Or pretty-printing by default
# rich.pretty.install()

# Intro

### Expected Knoledge

You should already know:

* Basic Python syntax
* Functions
* Basic classes (will cover quickly)
* Basic NumPy (will mention)
* Git - _CRITICAL FOR ANY SOFTWARE WORK!_

And we will be using notebooks in JupyterLab, but won't be talking about them much, except for one disclaimer: They are for teaching, testing, prototyping, and final analysis driving - they are _NOT_ a place to store significant amounts of code! Refactor into a file once you have developed something!

### What can you expect?

* I don't know what you know
* We don't have time to study any topic in depth.

So, we will move _fast_, and cover a _lot_.

You are not expected to able to master everything you see.

### Instead, you are expected to:

1. Know what is possible, so you know to look for it
2. Get pointers on where to look (lots of links!)
3. Refer back to this material later.

## About the author

[![Henryiii's github stats](https://github-readme-stats.vercel.app/api?username=henryiii)](https://github.com/anuraghazra/github-readme-stats)

https://iscinumpy.gitlab.io

Scikit-HEP admin, member of IRIS-HEP.

### Projects

cibuildwheel • pybind11 • boost-histogram • Hist • Vector • CLI11 • Plumbum • GooFit • Particle • DecayLanguage • Conda-Forge ROOT • POVM

### Intrests:

Packaging and building • Bindings • Building a HEP analysis toolchain in Python, JITable

## Theory: New features in programming

Programming is all about _orginization_. This is not always obvious, and has some odd consequences. Let's look at one: **new features remove functionality from the user**.

Don't believe me? Pick one. Let's go with an old, simple one you should already know: `goto` vs. loops (`for`/`while`). (This is my favorite example, even though Python thankfully came along late enough to not even have `goto` in the first place, except as an April fools joke.)


You have total power with goto! Jump from anywhere, too anywhere! You can recreate all loops with it, and more! So why are loops the newer, better feature?

####  goto ([partially hypothetical](https://github.com/snoack/python-goto) in Python)
```python
i = 0
label .start
print(f"Hi {i}")
i + 1
if i <= 10:
    goto .start
```
#### VS for loop:
    

In [ ]:
for i in range(10):
    print(f"Hi {i}")

A programmer has to spend time to recognise what is happing in the first example - in the second example, even a fairly new Python programmer will immediatly say "that prints 0 to 9". The second example lets you build more complex programs, because you are working at a 'higher level', humans tend to do better which high level concepts (while computers work up from low level).

# Python Object Model

## Everything in Python is an object!

> Okay, technically a `PyObject*` in CPython - we'll focus on CPython most of the time today.

An object is an "instance" of a "type", or a "class", which describes what that sort of object does.

> Types and basic objects have some optimizations in CPython, for speed and also to keep the language from being infinitely recursive - but they are still PyObject*'s.
>
> Also, "built-in" classes are a little special.

In [ ]:
v = 4

In [ ]:
v.bit_length()

In [ ]:
def f(x):
    return x**2

In [ ]:
# f, v = v, f
# What is the v now? f? How about the name?
# Note to instructor: restore f for the next part!

## Inspecting

You can inspect objects. There are lots of ways.

* In a juptyer notebook, use `object.<tab>` to bring up completions, shift tab for help.
* You can use dir(object) to see all attribututes (more or less)
* You can use help(object) or object? (IPython only) to see help
* You can `import inspect` and use the tools there
* You can install the rich library and use `rich.inspect()`

In [ ]:
# help(v)
# dir(v)
import inspect
print(inspect.getsourcefile(f))
print(inspect.getsource(f))
# WARNING! THIS DOES NOT ALWAYS WORK!

#### WARNING: You cannot *always* see the source of a function, so this is a user trick, not one to use in a library!

Python does a three stage procedure when interpreting. It convers source to bytecode (pyc files), then runs the bytecode in the interpreter. When loading a file that has been run before (or came from a wheel, more on that later), it only loads the bytecode if the source hasn't changed - the source is not reparsed. So inspect works by looking up the original file location. _But you can delete the original file and run from bytecode only!_. Don't do that, but you can. Also, you can run from a zip file, and the orignal file might not be openable. And, finally, when running live in a REPL, there may not be a source (it works in IPython for us, though).


In [ ]:
import rich
rich.inspect(f)

In [ ]:
rich.inspect(3)

## Mutablity

Many built-in objects are immutable, so it can hide the fact that Python does not have the concept of "pass by value"; the labels you see really are all pointing to PyObject*'s that are being managed by Python's garbage colllector.

If you write `x = y`, then x and y refer to the same object. Always. This can't be overriden. However, it's hard to see that if you can't mutate the value, so there's no "side effect" to this. Side effects only happen for mutable objects.

In [ ]:
x = 3
y = x
x += 1
print("What is {y}?")

Now, let's try a mutable object. Lists, sets, and dicts are the most notable mutable objects in `builtins`.

In [ ]:
x = [3]
y = x
x[0] += 1
print("What is {y = }?")

Why?

The problem was that when the object was imuatable, the inplace operator `+=` actually behaved like `x = x + 1`, which is a new object. When it was a mutable object (a list), then it was able to change it in-place.

#### Advanced aside: Why did this work?

Quick aside for advanced Pythonistas, this is tricky. `x[0]` returns an int. So why is this any different than before? Let's explore, using mock:

In [ ]:
from unittest.mock import MagicMock

ListProxy = MagicMock("ListProxy")
y = ListProxy()
y[0] += 1
rich.print(y.mock_calls)

You can see that this has special support for this syntax, it pulls out the item inside, and sets it, then stores it. There are other special syntax treatments in Python as well, all designed to make the language more friendly and powerful:



In [ ]:
1 < 2 < 3

In [ ]:
print(not 3 in {1, 2, 3})
print(3 not in {1, 2, 3})

Can we glean anything from the original example, though? Yes, assignment is special. There are only three forms of normal assignment:

```python
x = y = 1
x[y] = 2
x.y = 3
```

That's it. These are _not_ valid assignments:

```python
x(y) = 1    # There is no assignment for __call__
x + y = 2   # Arbitrary expressions are not allowed
(x = 2) + 2 # Can't be nested
# So on.
```

> This was limiting, so Python 3.8 added a nestable assignment, `:=` (walrus operator), though it was somewhat controversial. It does not work anywhere that normal `=` works, to avoid confusion.

Inplace assignment operators follow the same rules, so they are "special", and not allowed anywhere.

## Scope

Python has scope, but not in very many places. Functions and class definitions create scope, and modules have scope. (Generator expressions now have scope too). That's about it. So you can write this:

In [ ]:
if True:
    x = 3
print(x)

If `if` had scope, then x would not be accessable outside the if. This is simple and useful, but you have to be careful to stay clean and tidy. For example, if that `if` was `False`, this would suddenly break.

Because it shows up in so few places, and is so close to automatic, in can bite you once in a while if you don't keep it in mind:

In [ ]:
x = 1
def f():
    print(x)
    
f()
print(x)

But we will try an assignment:

In [ ]:
x = 1
def f():
    x = 2
    print(x)
    
f()
print(x)

So x in the function is not the same x out of the function now! (Try printing before assigning to it, or try changing it inplace. Even better, what happens if it is mutable?)

If this is really what you want, you can use `nonlocal` to access a variable one-scope-up, or `global` to declare a variable with global scope.

## Memory and the Garbage Collector

Python is a garbage collected language. All Python objects have a refcount, which tells the garbage collector how many "labels" or how many ways you can access that object. When you can't get to the object anymore, the garbage collector _has the right to_ remove the object. It _usually_, depending on settings, runs rougly once per line, but it doesn't have to.

As a consequence, _never_ depend on an object being deleted to perform some action at some time. More on that later, when we cover context managers.

Let's look at it:

## Importing

Objects are _very_ powerful, and you can harness this power for your own designs!

# Object Oriented Programming

## Writing a class

Writing a class should be second nature, and very natural to you - just as natural as writing a function!

In [ ]:
def f(x):
    return x**2

print(f"{f(3) = }")

In [ ]:
class F:
    def __call__(self, x):
        return x**2
    
f = F()
print(f"{f(3) = }")


# Errors and catching them
# Generators (Iterators)
# Dectorators
# Context Managers
# Static Type Hinting
# Packaging and Files

# PyTest: Unit Testing
# NumPy: You should know this one
# Pandas: DataFrames for Python
# Numba: JIT for Speed!
# Pybind11: Use C++ libraries
# Code Quality and CI